In [ ]:
import os
from gpuutils import GpuUtils
GpuUtils.allocate(gpu_count=1, framework='keras')

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True) 

import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
np.set_printoptions(precision=3, suppress=True)
from sklearn.preprocessing import MinMaxScaler


In [ ]:
df = pd.read_csv('/mnt/md0/aholmberg/data/raytrace_samples_angle.csv')

#df['launch_angle'] = np.arctan2(df['launch_vec_r'].to_numpy(), df['launch_vec_z'].to_numpy())
#df['recieve_angle'] = np.arctan2(df['recieve_vec_r'].to_numpy(), df['recieve_vec_z'].to_numpy())
#df.to_csv('/mnt/md0/aholmberg/data/raytrace_samples_angle.csv', index=False)
df.head()

In [ ]:
sc_pos_r = df['source_pos_r'].to_numpy().astype(np.float32)
sc_pos_z = df['source_pos_z'].to_numpy().astype(np.float32)
ant_pos_z = df['antenna_pos_z'].to_numpy().astype(np.float32)
features = np.stack((sc_pos_r, sc_pos_z, ant_pos_z), axis=1)

path_length = df['path_length'].to_numpy().astype(np.float32)
travel_time = df['travel_time'].to_numpy().astype(np.float32)

#launch_vec_r = df['launch_vec_r'].to_numpy().astype(np.float32)
#launch_vec_z = df['launch_vec_z'].to_numpy().astype(np.float32)
#launch = np.stack((launch_vec_r, launch_vec_z), axis=1)

#recieve_vec_r = df['recieve_vec_r'].to_numpy().astype(np.float32)
#recieve_vec_z = df['recieve_vec_z'].to_numpy().astype(np.float32)
#recieve = np.stack((recieve_vec_r, recieve_vec_z), axis=1)

launch = df['launch_angle'].to_numpy().astype(np.float32)
recieve = df['recieve_angle'].to_numpy().astype(np.float32)

typ = df['type'].to_numpy()
print(launch.shape, recieve.shape, travel_time.shape, path_length.shape)

In [ ]:
features  = features[0::2,:]
features_train = features[:int(features.shape[0]*0.8)]
features_test = features[int(features.shape[0]*0.8):]

temp1 = travel_time[0::2]
temp2 = travel_time[1::2]
out_time = np.stack((temp1, temp2), axis=1)
out_time_train = out_time[:int(out_time.shape[0]*0.8),:]
out_time_test = out_time[int(out_time.shape[0]*0.8):,:]


temp1 = path_length[0::2]
temp2 = path_length[1::2]
out_length = np.stack((temp1, temp2), axis=1)
out_length_train = out_length[:int(out_length.shape[0]*0.8),:]
out_length_test = out_length[int(out_length.shape[0]*0.8):,:]

temp1 = launch[0::2]
temp2 = launch[1::2]
out_launch = np.stack((temp1, temp2), axis=1)
out_launch_train = out_launch[:int(out_launch.shape[0]*0.8),:]
out_launch_test = out_launch[int(out_launch.shape[0]*0.8):,:]

temp1 = recieve[0::2]
temp2 = recieve[1::2]
out_recieve = np.stack((temp1, temp2), axis=1)
out_recieve_train = out_recieve[:int(out_recieve.shape[0]*0.8),:]
out_recieve_test = out_recieve[int(out_recieve.shape[0]*0.8):,:]

"""
out1_train = out1[:int(out1.shape[0]*0.8)]
out1_test = out1[int(out1.shape[0]*0.8):]

out2_train = out2[:int(out2.shape[0]*0.8)]
out2_test = out2[int(out2.shape[0]*0.8):]
"""
print(out_time.shape, out_length.shape, out_launch.shape, out_recieve.shape)

In [ ]:
temp1 = typ[0::2]
temp2 = typ[1::2]
print(temp1, temp2, typ)
print(temp1.shape, temp2.shape, typ.shape)
unique, counts = np.unique(temp1, return_counts=True)
a = dict(zip(unique, counts))
unique, counts = np.unique(temp2, return_counts=True)
b = dict(zip(unique, counts))
print(a, b)

In [ ]:
unique, counts = np.unique(features_train[:,0], return_counts=True)
#c = dict(zip(unique, counts))
print(max(counts))

In [ ]:
scaler_features = MinMaxScaler(feature_range=(0,1))
scaler_features.fit(features_train)
norm_features_train = scaler_features.transform(features_train)
norm_features_test = scaler_features.transform(features_test)

scaler_time = MinMaxScaler(feature_range=(0,1))
scaler_time.fit(out_time_train)
norm_time_train = scaler_time.transform(out_time_train)
norm_time_test = scaler_time.transform(out_time_test)

scaler_length = MinMaxScaler(feature_range=(0,1))
scaler_length.fit(out_length_train)
norm_length_train = scaler_length.transform(out_length_train)
norm_length_test = scaler_length.transform(out_length_test)

scaler_launch = MinMaxScaler(feature_range=(0,1))
scaler_launch.fit(out_launch_train)
norm_launch_train = scaler_launch.transform(out_launch_train)
norm_launch_test = scaler_launch.transform(out_launch_test)

scaler_recieve = MinMaxScaler(feature_range=(0,1))
scaler_recieve.fit(out_recieve_train)
norm_recieve_train = scaler_recieve.transform(out_recieve_train)
norm_recieve_test = scaler_recieve.transform(out_recieve_test)

#normalize = layers.Normalization()
#normalize.adapt(features_train)

In [ ]:
"""
inputs = keras.Input(shape=(3, ))
#x = normalize(inputs)
x = layers.Dense(32, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x1 = layers.Dense(64, activation='relu')(x)
x2 = layers.Dense(64, activation='relu')(x)
x3 = layers.Dense(64, activation='relu')(x)
x4 = layers.Dense(64, activation='relu')(x)
x1 = layers.Dense(32, activation='relu')(x1)
x2 = layers.Dense(32, activation='relu')(x2)
x3 = layers.Dense(32, activation='relu')(x3)
x4 = layers.Dense(32, activation='relu')(x4)
output_time = layers.Dense(2, activation='relu')(x1)
output_length = layers.Dense(2, activation='relu')(x2)
output_launch = layers.Dense(4, activation='relu')(x3)
output_recieve = layers.Dense(4, activation='relu')(x4)
"""


In [ ]:
#model = keras.Model(inputs=inputs, outputs=[output_time, output_length, output_launch, output_recieve], name="full_model")
from raytrace_model_def import Fc_model
activation = keras.layers.ReLU()
layers = [128, 256, 512]
model = Fc_model(layers, activation)

In [ ]:
#keras.utils.plot_model(model, "travel_time_model.png")

In [ ]:
model.build((1,3))
print(model.summary())

In [ ]:
model.compile(optimizer='adam', loss='mse')

In [ ]:
y = np.concatenate((norm_time_train, norm_length_train, norm_launch_train, norm_recieve_train), axis=1)
print(norm_features_train.shape)

In [ ]:
#model.fit(norm_features_train, [norm_time_train, norm_length_train, norm_launch_train, norm_recieve_train], epochs=5)
model.fit(norm_features_train, y, epochs=1)

In [ ]:
"""
y_test = model(norm_features_test)
y_time = y_test[0].numpy()
y_length = y_test[1].numpy()
y_launch = y_test[2].numpy()
y_recieve = y_test[3].numpy()
"""
y_test = model(norm_features_test).numpy()
print(y_test.shape)
y_time = y_test[:,:2]
y_length = y_test[:,2:4]
y_launch = y_test[:,4:6]
y_recieve = y_test[:,6:8]


In [ ]:
import matplotlib.pyplot as plt
print(y_time.shape, norm_time_test.shape)

In [ ]:
plt.scatter(norm_time_test[:,0], y_time[:,0])
plt.xlabel('Analytically calculated travel time (ns)', size=14)
plt.ylabel('Network prediction (ns)', size=14)
#plt.savefig('model_pred_1.svg')

In [ ]:
plt.scatter(norm_time_test[:,1], y_time[:,1])
plt.xlabel('Analytically calculated travel time (ns)', size=14)
plt.ylabel('Network prediction (ns)', size=14)
#plt.savefig('model_pred_2.svg')

In [ ]:
y_time_inv = scaler_time.inverse_transform(y_time)
diff_time = out_time_test - y_time_inv
print(f'Mean of difference true-pred {np.mean(diff_time[:,0]):.4f} Std {np.std(diff_time[:,0]):.4f}')
print(f'Mean of difference true-pred {np.mean(diff_time[:,1]):.4f} Std {np.std(diff_time[:,1]):.4f}')

In [ ]:
y_length_inv = scaler_length.inverse_transform(y_length)
diff_length = out_length_test - y_length_inv
print(f'Mean of difference true-pred {np.mean(diff_length[:,0]):.4f} Std {np.std(diff_length[:,0]):.4f}')
print(f'Mean of difference true-pred {np.mean(diff_length[:,1]):.4f} Std {np.std(diff_length[:,1]):.4f}')

In [ ]:
y_launch_inv = scaler_launch.inverse_transform(y_launch)
diff_launch = out_launch_test - y_launch_inv
print(f'Mean of difference true-pred {np.mean(diff_launch[:,0]):.4f} Std {np.std(diff_launch[:,0]):.4f}')
print(f'Mean of difference true-pred {np.mean(diff_launch[:,1]):.4f} Std {np.std(diff_launch[:,1]):.4f}')

In [ ]:
y_recieve_inv = scaler_recieve.inverse_transform(y_recieve)
diff_recieve = out_recieve_test - y_recieve_inv
print(f'Mean of difference true-pred {np.mean(diff_recieve[:,0]):.4f} Std {np.std(diff_recieve[:,0]):.4f}')
print(f'Mean of difference true-pred {np.mean(diff_recieve[:,1]):.4f} Std {np.std(diff_recieve[:,1]):.4f}')

In [ ]:
print(y_length_inv)

In [ ]:
#from radiotools import plthelpers as php
#fix, ax = php.get_histogram(diff[:,1], bins=50)
#plt.show()
print(norm_time_train)